# crop tank vids

Edit tank vides to remove sections without a fish. Create new videos, with one video per fish appearance.

In [4]:
from scipy.signal import medfilt
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import numpy as np
import subprocess
import pickle
import random
import time
import cv2
import os


# settings
root_dir = r'Z:\locker\ShareData\Chin_for_Dillon'
bg_frame_num = 100    # frames to include in background calculation
diff_dt = .1          # (seconds) time between background frame computation
thresh = 3.0;         # (z scored pixels) difference between frame and background frame for fish detection
buffer = 1;           # (s) time before and after fish appearance to keep
med_filtering = 1.0;  # (s) median filter the thresholded signal to remove short periods
bit_rate = 12;        # (megabits per second)




In [ ]:
vid_folders = [f.path for f in os.scandir(root_dir) if f.is_dir()]
for vid_folder in vid_folders:
    sub_folders = [f.path for f in os.scandir(vid_folder) if f.is_dir()]
    for sub_folder in sub_folders:
        crop_video(os.path.join(sub_folder, 'videoEOD.avi'))


------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\20200903\time_h11m28s22\videoEOD.avi -------
cropping from 22.7 to 29.7 seconds (000)

------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\20200903\time_h11m30s47\videoEOD.avi -------
cropping from 15.0 to 19.7 seconds (000)

------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\20200903\time_h11m31s58\videoEOD.avi -------
cropping from 19.2 to 24.9 seconds (000)

------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\20200903\time_h11m32s31\videoEOD.avi -------
cropping from 23.4 to 29.3 seconds (000)

------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\20200903\time_h11m33s4\videoEOD.avi -------
cropping from 111.4 to 115.1 seconds (000)

------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\20200903\time_h11m35s11\videoEOD.avi -------
WARNING! Fish not detected!

------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\20200903\time_h11m38s9\videoEOD.avi -------
cropping from 38.3 to 42.4 seconds (000)
cropping from 51.4 to 59.1

In [28]:
def crop_video(video):

    # initialize video
    print(f'\n------- ANALYZING {video} -------')
    vid = cv2.VideoCapture(video)
    n_frames_approx = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))  # approximate
    fps = int(vid.get(cv2.CAP_PROP_FPS))
    frame_delta = round(diff_dt * fps)
    name, ext = os.path.splitext(video)

    # compute background
    # (median across randomly sampled frames)
    bg_frame_nums = random.sample(range(n_frames_approx), bg_frame_num)
    bg_frame_nums.sort()

    bg_frames = []
    for i in bg_frame_nums:
        vid.set(1, i)  # using `set` instead of reading through all the frames is less accurate but much faster
        _, frame = vid.read();
        bg_frames.append(frame)
    vid.release()
    bg = np.median(np.array(bg_frames), axis=0)

    # compute differences between frames and background
    vid = cv2.VideoCapture(video)  # reinitialize
    got_frame = True
    diffs = []
    delta_counter = frame_delta-1  # only compute background every delta frames
    n_frames = 0

    while got_frame:
        got_frame, frame = vid.read();
        if got_frame: n_frames += 1
        delta_counter += 1
        if got_frame and delta_counter==frame_delta:
            diffs.append(np.abs(frame - bg).mean())
            delta_counter = 0
    vid.release()
    diffs = np.array(diffs)

    # find fish appearances
    t = np.arange(0, diffs.shape[0]*(frame_delta/fps), frame_delta/fps)
    threshed = diffs > thresh
    kernel_sz = round(med_filtering*fps/frame_delta)
    kernel_sz = kernel_sz - kernel_sz%2 + 1  # make odd
    threshed = medfilt(threshed, kernel_sz)  # median filtering to debounce

    # plot
    fig = plt.figure(figsize=(10,2.5))
    ax = plt.axes(xlabel='time (s)', ylabel='frame diffs')
    ax.plot(t, diffs);
    ax.plot(t, threshed*np.ptp(diffs) + min(diffs));
    plt.savefig(os.path.join(os.path.dirname(video), 'cropping.png'), facecolor=(1,1,1,1))
    plt.savefig(os.path.join(root_dir, 'cropping', '{}_{}_cropping.png'.format(
        os.path.basename(str(Path(video).parents[1])),
        os.path.basename(str(Path(video).parents[0])) )), facecolor=(1,1,1,1))
    
    # create vids for each epoch
    if not threshed.any():
        print('WARNING! Fish not detected!')
        return

    start_inds = np.where(np.diff(threshed)==1)[0]+1
    end_inds   = np.where(np.diff(threshed)==-1)[0]+1
    if start_inds[0] < end_inds[0]:
        np.insert(start_inds, 0, 0)
    if start_inds[-1] > end_inds[-1]:
        end_inds.append(len(diffs)-1)

    # delete old cropped video if exists
    old_name = '{}_cropped{}'.format(name, ext)
    if os.path.exists(old_name):
        print('deleting {}'.format(old_name))
        os.remove(old_name)

    # create new cropped videos
    for i, (start_ind, end_ind) in enumerate(zip(start_inds, end_inds)):
        output_name = '{}_cropped{:03d}{}'.format(name, i, ext)
        start = max(0, start_ind*(frame_delta/fps) - buffer)
        end   = min(n_frames/fps, end_ind*(frame_delta/fps) + buffer)
        print('cropping from {:.1f} to {:.1f} seconds ({:03d})'.format(start, end, i))
        command = 'ffmpeg -i {} -ss {:.2f} -t {:.2f} -vb {}M -y -loglevel panic -stats {}'.format(
            video, start, end-start, bit_rate, output_name)
        os.system(command)
